In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

deathWoman = np.array(pd.read_excel('../../data/여성사망률추이.xlsx').set_index(['연도']).iloc[:43,:100])
deathMan = np.array(pd.read_excel('../../data/남성사망률추이.xlsx').set_index(['연도']).iloc[:43,:100])
dataOriDf = pd.read_excel('../../data/데이터정리3.xlsx')
ageDataOriDf = pd.read_excel('../../data/의사연령별분포.xlsx')
npAgeData = np.array(pd.read_excel('../../data/의사연령별분포.xlsx'))

In [2]:
ageDataOriDf.iloc[:,1:]

,총/합계,총/~29,총/30~39,총/40~49,총/50~59,총/60~,남/합계,남/~29,남/30~39,남/40~49,...,남비율/30~39,남비율/40~49,남비율/50~59,남비율/60~,여비율/합계,여비율/~29,여비율/30~39,여비율/40~49,여비율/50~59,여비율/60~
0,82685,839,23488,27826,19466,11066,64371,561,14723,22244,...,0.178061,0.269021,0.204160,0.120481,0.221491,0.003362,0.106005,0.067509,0.031263,0.013352
1,85073,1963,25100,27820,19381,10809,65989,1356,15836,22270,...,0.186146,0.261775,0.197430,0.114384,0.224325,0.007135,0.108895,0.065238,0.030386,0.012671
2,88945,3459,27682,27933,19357,10514,68330,2297,17412,22366,...,0.195761,0.251459,0.188600,0.106583,0.231772,0.013064,0.115465,0.062589,0.029029,0.011625
3,91197,5078,28738,27892,19258,10231,69677,3258,18114,22374,...,0.198625,0.245337,0.183065,0.101275,0.235973,0.019957,0.116495,0.060506,0.028104,0.010910
4,93484,6955,29528,27928,19203,9870,71066,4328,18808,22359,...,0.201190,0.239175,0.178170,0.095364,0.239806,0.028101,0.114672,0.059572,0.027245,0.010216
5,96180,8795,30729,27938,19154,9564,72818,5272,19852,22420,...,0.206405,0.233105,0.173030,0.089748,0.242899,0.036629,0.113090,0.057372,0.026118,0.009690


In [3]:
pd.set_option('display.max_row', 1200)
pd.set_option('display.max_columns', 100)

#1977년 합격자부터 2019년 합격자
dataDf = dataOriDf[['년도','의대졸합격/남', '의대졸합격/여', '의전졸합격/남', '의전졸합격/여', '불합합격/남', '불합합격/여']]
dataDf = dataDf[27:70]

npData = np.around(np.array(dataDf)[:,1:7])
npData

array([[ 932.,  155.,    0.,    0.,   94.,   17.],
       [ 963.,  154.,    0.,    0.,   70.,   12.],
       [ 953.,  156.,    0.,    0.,   95.,   15.],
       [1030.,  178.,    0.,    0.,   98.,   16.],
       [1026.,  132.,    0.,    0.,  149.,   26.],
       [1220.,  111.,    0.,    0.,   59.,    8.],
       [1187.,  150.,    0.,    0.,   61.,    6.],
       [1032.,  188.,    0.,    0.,  294.,   37.],
       [1242.,  181.,    0.,    0.,  139.,   25.],
       [1656.,  302.,    0.,    0.,   94.,   14.],
       [2241.,  304.,    0.,    0.,   46.,    8.],
       [2157.,  391.,    0.,    0.,  133.,   18.],
       [2194.,  526.,    0.,    0.,  180.,   33.],
       [1983.,  493.,    0.,    0.,  260.,   62.],
       [2195.,  535.,    0.,    0.,  187.,   47.],
       [2116.,  550.,    0.,    0.,  199.,   48.],
       [2400.,  616.,    0.,    0.,   75.,   20.],
       [2282.,  522.,    0.,    0.,   81.,   21.],
       [1602.,  354.,    0.,    0.,  692.,  158.],
       [1321.,  337.,    0.,   

# 시그모이드 변형 모델

In [4]:
# 분류모델함수 생성
# x - x값, sp - 시작점, xl - x축 조정, yl - y축 조정
def sigmoid_function(x, sp, xl, yl):
    
    
    
    result = 1 / ( yl+np.exp((x-(sp+(5/xl)))*xl) )
    
    for i in range(np.size(x)):
        if x[i]<sp:
           result[i]=0
        if x[i]>(sp+(10/xl)):
           result[i]=0
            
    return result

# 분류모델 확률질량함수 생성
def makeSigmoidModel(sp, xl, yl):
    x = np.arange(1, 101, 1)
    y = sigmoid_function(x, sp, xl, yl)
    sum = np.sum(y)
    z = y/sum
    
    return z



def clusterAgeModel(rate,st,end):
    npData = np.zeros(100)
    
    for i in range(st,end):
        npData[i] = (rate**(i-end))-1  
        
    sumValue = np.sum(npData)
    npRateData = npData / sumValue
    return npRateData

In [5]:
pd.DataFrame(clusterAgeModel(0.7,25,40)).T
np.array(range(30,50))

array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
       47, 48, 49])

# 사망률 적용

In [6]:
def makeAlivePerson(npData,year):
    
    deathRate = np.array([deathMan[year], deathWoman[year]])
    resultData = npData - np.around((npData*deathRate))
    
    return resultData

# 은퇴율 적용

In [7]:
def makeWorkPerson(npData,tuningSet):
    valueList = np.zeros([2,100])
    c1 = tuningSet[2][0]/((tuningSet[0][0]**(100-tuningSet[1][0]))-1)
    c2 = tuningSet[2][1]/((tuningSet[0][1]**(100-tuningSet[1][1]))-1)
    for i in range(100):
        result1 = ((tuningSet[0][0]**(i-tuningSet[1][0]))-1)*c1
        result2 = ((tuningSet[0][1]**(i-tuningSet[1][1]))-1)*c2
        valueList[0][i] = 0 if result1<0 else 1 if result1>1 else  result1
        valueList[1][i] = 0 if result2<0 else 1 if result2>1 else  result2

    result = npData - np.around(npData*valueList)
    
    return result

In [8]:
def makeWorkPerson1(tuningSet):
    valueList = np.zeros([2,100])
    c1 = 1/((tuningSet[0][0]**(100-tuningSet[1][0]))-1)
    c2 = 1/((tuningSet[0][1]**(100-tuningSet[1][1]))-1)
    for i in range(100):
        result1 = ((tuningSet[0][0]**(i-tuningSet[1][0]))-1)*c1
        result2 = ((tuningSet[0][1]**(i-tuningSet[1][1]))-1)*c2
        valueList[0][i] = 0 if result1<0 else 1 if result1>1 else  result1
        valueList[1][i] = 0 if result2<0 else 1 if result2>1 else  result2

    
    return valueList

pd.DataFrame(makeWorkPerson1([[1.5,1.5],[25,25]]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.105457e-14,7.763643e-14,1.475092e-13,2.523184e-13,4.095322e-13,6.453529e-13,9.990839e-13,1.529680e-12,2.325575e-12,3.519417e-12,5.310181e-12,7.996325e-12,1.202554e-11,1.806937e-11,2.713511e-11,4.073372e-11,6.113163e-11,9.172850e-11,1.376238e-10,2.064668e-10,3.097312e-10,4.646278e-10,6.969728e-10,1.045490e-09,1.568266e-09,2.352431e-09,3.528677e-09,5.293047e-09,7.939601e-09,1.190943e-08,1.786418e-08,2.679630e-08,4.019448e-08,6.029176e-08,9.043766e-08,1.356565e-07,2.034848e-07,3.052273e-07,4.578409e-07,6.867614e-07,0.000001,0.000002,0.000002,0.000003,0.000005,0.000008,0.000012,0.000018,0.000026,0.00004,0.000059,0.000089,0.000134,0.0002,0.000301,0.000451,0.000677,0.001015,0.001522,0.002284,0.003425,0.005138,0.007707,0.011561,0.017342,0.026012,0.039018,0.058528,0.087791,0.131687,0.197531,0.296296,0.444444,0.666667
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.105457e-14,7.763643e-14,1.475092e-13,2.523184e-13,4.095322e-13,6.453529e-13,9.990839e-13,1.529680e-12,2.325575e-12,3.519417e-12,5.310181e-12,7.996325e-12,1.202554e-11,1.806937e-11,2.713511e-11,4.073372e-11,6.113163e-11,9.172850e-11,1.376238e-10,2.064668e-10,3.097312e-10,4.646278e-10,6.969728e-10,1.045490e-09,1.568266e-09,2.352431e-09,3.528677e-09,5.293047e-09,7.939601e-09,1.190943e-08,1.786418e-08,2.679630e-08,4.019448e-08,6.029176e-08,9.043766e-08,1.356565e-07,2.034848e-07,3.052273e-07,4.578409e-07,6.867614e-07,0.000001,0.000002,0.000002,0.000003,0.000005,0.000008,0.000012,0.000018,0.000026,0.00004,0.000059,0.000089,0.000134,0.0002,0.000301,0.000451,0.000677,0.001015,0.001522,0.002284,0.003425,0.005138,0.007707,0.011561,0.017342,0.026012,0.039018,0.058528,0.087791,0.131687,0.197531,0.296296,0.444444,0.666667


# 출신 연령별 분류모델

In [9]:
def makeArrayUseModel(tuningList):
    model1 = clusterAgeModel(tuningList[0][0], tuningList[1][0], tuningList[2][0]) # 의대 남
    model2 = clusterAgeModel(tuningList[0][1], tuningList[1][1], tuningList[2][1]) # 의대 여
    model3 = clusterAgeModel(tuningList[0][2], tuningList[1][2], tuningList[2][2]) # 의전원 남
    model4 = clusterAgeModel(tuningList[0][3], tuningList[1][3], tuningList[2][3]) # 의전원 여
    model5 = clusterAgeModel(tuningList[0][4], tuningList[1][4], tuningList[2][4]) # 재시험 남
    model6 = clusterAgeModel(tuningList[0][5], tuningList[1][5], tuningList[2][5]) # 재시험 여
    
    resultData =  np.array([model1, model2, model3, model4, model5, model6])
    
    return resultData

# 신규인원 생성

In [10]:
def makeNewPerson(npData, tuningSet):
    oldSize = 100
    yearSize = len(npData)
    modelSize = len(npData[0])
    
    #모델 적용 배열
    applyModelArray = np.zeros((yearSize,modelSize,oldSize))

    #신규인원 배열
    newPersonArray = np.zeros((yearSize,2,oldSize))

    
    modelAry = makeArrayUseModel(tuningSet)
    
    for i in range(yearSize):
        for j in range(modelSize):
            applyModelArray[i][j] = np.around(modelAry[j]*np.around(npData[i][j]))

            if j%2==0:
                newPersonArray[i][0] += applyModelArray[i][j]
            else:
                newPersonArray[i][1] += applyModelArray[i][j]
    
    resultData = newPersonArray
    
    return resultData

# 최종인원 도출

In [11]:
def shiftOld(personArray):
    dataArray = np.zeros((2,len(personArray[0])))
    dataArray[0] = np.roll(personArray[0], 1)  
    dataArray[1] = np.roll(personArray[1], 1)
    dataArray[0][0] = 0
    dataArray[1][0] = 0
    
    return dataArray

def makeResultPersonArray(newPersonArray, tuningSet):
    #누적인원 배열
    
    sizeArray = list(np.shape(newPersonArray))
    yearSize = sizeArray[0]
    resultPersonArray = np.zeros(sizeArray)

    for i in range(yearSize):
        if i>0:
            # 1살 올리기 / shiftData -> 2 x 100
            shiftData = shiftOld(resultPersonArray[i-1])
            # 사망률 적용 / aliveData -> 2 x 100
            aliveData = makeAlivePerson(shiftData, i)
            # 은퇴율 적용 / workData -> 2 x 100
            workData = makeWorkPerson(aliveData, tuningSet)
            # 최종 계산
            resultPersonArray[i] =  workData + newPersonArray[i]
        else:
            resultPersonArray[i] =  newPersonArray[i]
            
            
    return resultPersonArray

# 연령별 합산

In [12]:
def sumPeopleUseAge(npData):
    yearSize = len(npData)
    
    resultData = np.zeros([yearSize, 3, 12])
    
    for i in range(yearSize):
        for j in range(2):
            # 남/여 소계 계산
            resultData[i][j][0] = np.sum(npData[i][j])
            resultData[i][j][1] = np.sum(npData[i][j][0:30])
            resultData[i][j][2] = np.sum(npData[i][j][30:40])
            resultData[i][j][3] = np.sum(npData[i][j][40:50])
            resultData[i][j][4] = np.sum(npData[i][j][50:60])
            resultData[i][j][5] = np.sum(npData[i][j][60:])
            
            # 남/여 비율 계산
            for n in range(6):
                resultData[i][j][6+n] = resultData[i][j][n]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
        
        for j in range(6):
            # 합계 계산
            resultData[i][2][j] = resultData[i][0][j] + resultData[i][1][j]  
            # 합계 비율 계산
            resultData[i][2][6+j] = resultData[i][2][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
            # 남/여 비율 계산
            resultData[i][0][6+j] = resultData[i][0][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
            resultData[i][1][6+j] = resultData[i][1][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0


    return resultData

# 코스트 계산

In [13]:
def calculateCost(npData):   
    #totalData = ageDataOriDf.iloc[:,1:7]
    #manData = ageDataOriDf.iloc[:,7:13]
    #womanData = ageDataOriDf.iloc[:,13:19]
    #totalRateData = ageDataOriDf.iloc[:,19:25]
    #manRateData = ageDataOriDf.iloc[:,25:31]
    #twomanRateData = ageDataOriDf.iloc[:,31:37]
    resultData = np.zeros([7,6])

    for i in range(6):
        manNumCost = np.sum(np.abs(npData[39-i][0][1:6] - npAgeData[5-i][8:13]))
        womanNumCost = np.sum(np.abs(npData[39-i][1][1:6] - npAgeData[5-i][14:19]))
        totalNumCost = np.sum(np.abs(npData[39-i][2][1:6] - npAgeData[5-i][2:7]))
        
        manRateCost = np.sum(np.abs(npData[39-i][0][7:12] - npAgeData[5-i][26:31]))
        womanRateCost = np.sum(np.abs(npData[39-i][1][7:12] - npAgeData[5-i][32:37]))

        sumRateCost = manRateCost + womanRateCost
        
        npCostArray = np.array([manNumCost, womanNumCost, totalNumCost, 
                    manRateCost, womanRateCost, sumRateCost])
        
        resultData[i] = npCostArray
    
    for i in range(6):
        resultData[6][i] = sum(resultData[:6,i])
    
    return resultData

In [14]:
def getCost(npData, tuningSet):
    newPerson = makeNewPerson(npData, tuningSet[0])
    resultPerson = makeResultPersonArray(newPerson,tuningSet[1])
    # sumData -> 43 x 3 x 12
    sumData = sumPeopleUseAge(resultPerson)
    costData = calculateCost(sumData)
    
    return [sumData, costData]

In [44]:
npData = np.around(np.array(dataDf)[:,1:7])
npData

tuningSetAgeRate = [[0.9,0.9,0.1,0.1,0.9,0.9],[26,26,28,28,27,27],[40, 40, 42, 42, 41, 41]]
tuningSetRetireRate = [[1.1, 1.1],[30, 30],[0.8, 0.8]]
tuningSet = [tuningSetAgeRate, tuningSetRetireRate]
print(tuningSet)
%time data = getCost(npData,tuningSet)

[[[0.9, 0.9, 0.1, 0.1, 0.9, 0.9], [26, 26, 28, 28, 27, 27], [40, 40, 42, 42, 41, 41]], [[1.1, 1.1], [30, 30], [0.8, 0.8]]]
Wall time: 18 ms


In [45]:
dfResultCost = pd.DataFrame(data[1])
dfResultCost.columns = ["남성오차", "여성오차", "남여총계오차","남비율오차", "여비율오차", "비율총계오차"]
dfResultCost.index = ["2011", "2012", "2013","2014", "2015", "2016", "합계"]
dfResultCost

,남성오차,여성오차,남여총계오차,남비율오차,여비율오차,비율총계오차
2011,7415.0,7039.0,11456.0,0.076371,0.073222,0.149593
2012,5703.0,5220.0,8261.0,0.061938,0.055811,0.117749
2013,5492.0,4534.0,6394.0,0.060071,0.049584,0.109655
2014,7671.0,3859.0,8492.0,0.086176,0.043376,0.129552
2015,12388.0,3315.0,12855.0,0.149753,0.034997,0.184750
2016,16357.0,3462.0,17753.0,0.201417,0.037838,0.239256
합계,55026.0,27429.0,65211.0,0.635727,0.294828,0.930555


In [46]:
dfResult = pd.DataFrame(data[0][0])
indexList = []
indexList.append('1977 남성')
indexList.append('1977 여성')
indexList.append('1977 합계')
    
for i in range(42):
    dfResult = pd.concat([dfResult,pd.DataFrame(data[0][i+1])])
    indexList.append(str(i+1978)+' 남성')
    indexList.append(str(i+1978)+' 여성')
    indexList.append(str(i+1978)+' 합계')

dfResult.columns = ['합계', '0~29', '30~39', '40~49', '50~59', '60~', '합계/비율', '0~29/비율','30~39/비율', '40~49/비율', '50~59/비율', '60~']
dfResult.index = indexList
dfResult

,합계,0~29,30~39,40~49,50~59,60~,합계/비율,0~29/비율,30~39/비율,40~49/비율,50~59/비율,60~
1977 남성,1026.0,563.0,462.0,1.0,0.0,0.0,0.856427,0.469950,0.385643,0.000835,0.000000,0.000000
1977 여성,172.0,95.0,77.0,0.0,0.0,0.0,0.143573,0.079299,0.064274,0.000000,0.000000,0.000000
1977 합계,1198.0,658.0,539.0,1.0,0.0,0.0,1.000000,0.549249,0.449917,0.000835,0.000000,0.000000
1978 남성,2058.0,1018.0,1033.0,7.0,0.0,0.0,0.858932,0.424875,0.431135,0.002922,0.000000,0.000000
1978 여성,338.0,168.0,169.0,1.0,0.0,0.0,0.141068,0.070117,0.070534,0.000417,0.000000,0.000000
1978 합계,2396.0,1186.0,1202.0,8.0,0.0,0.0,1.000000,0.494992,0.501669,0.003339,0.000000,0.000000
1979 남성,3100.0,1342.0,1731.0,27.0,0.0,0.0,0.858964,0.371848,0.479634,0.007481,0.000000,0.000000
1979 여성,509.0,220.0,285.0,4.0,0.0,0.0,0.141036,0.060959,0.078969,0.001108,0.000000,0.000000
1979 합계,3609.0,1562.0,2016.0,31.0,0.0,0.0,1.000000,0.432807,0.558603,0.008590,0.000000,0.000000
1980 남성,4218.0,1555.0,2597.0,66.0,0.0,0.0,0.857317,0.316057,0.527846,0.013415,0.000000,0.000000


In [220]:
len(dfResult)

129

In [221]:
npAgeData = np.array(pd.read_excel('../../data/의사연령별분포.xlsx'))
pd.DataFrame(npAgeData).set_index(0)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011.0,82685.0,839.0,23488.0,27826.0,19466.0,11066.0,64371.0,561.0,14723.0,22244.0,16881.0,9962.0,18314.0,278.0,8765.0,5582.0,2585.0,1104.0,1.0,0.010147,0.284066,0.336530,0.235424,0.133833,0.778509,0.006785,0.178061,0.269021,0.204160,0.120481,0.221491,0.003362,0.106005,0.067509,0.031263,0.013352
2012.0,85073.0,1963.0,25100.0,27820.0,19381.0,10809.0,65989.0,1356.0,15836.0,22270.0,16796.0,9731.0,19084.0,607.0,9264.0,5550.0,2585.0,1078.0,1.0,0.023074,0.295041,0.327013,0.227816,0.127056,0.775675,0.015939,0.186146,0.261775,0.197430,0.114384,0.224325,0.007135,0.108895,0.065238,0.030386,0.012671
2013.0,88945.0,3459.0,27682.0,27933.0,19357.0,10514.0,68330.0,2297.0,17412.0,22366.0,16775.0,9480.0,20615.0,1162.0,10270.0,5567.0,2582.0,1034.0,1.0,0.038889,0.311226,0.314048,0.217629,0.118208,0.768228,0.025825,0.195761,0.251459,0.188600,0.106583,0.231772,0.013064,0.115465,0.062589,0.029029,0.011625
2014.0,91197.0,5078.0,28738.0,27892.0,19258.0,10231.0,69677.0,3258.0,18114.0,22374.0,16695.0,9236.0,21520.0,1820.0,10624.0,5518.0,2563.0,995.0,1.0,0.055682,0.315120,0.305843,0.211169,0.112186,0.764027,0.035725,0.198625,0.245337,0.183065,0.101275,0.235973,0.019957,0.116495,0.060506,0.028104,0.010910
2015.0,93484.0,6955.0,29528.0,27928.0,19203.0,9870.0,71066.0,4328.0,18808.0,22359.0,16656.0,8915.0,22418.0,2627.0,10720.0,5569.0,2547.0,955.0,1.0,0.074398,0.315862,0.298746,0.205415,0.105580,0.760194,0.046297,0.201190,0.239175,0.178170,0.095364,0.239806,0.028101,0.114672,0.059572,0.027245,0.010216
2016.0,96180.0,8795.0,30729.0,27938.0,19154.0,9564.0,72818.0,5272.0,19852.0,22420.0,16642.0,8632.0,23362.0,3523.0,10877.0,5518.0,2512.0,932.0,1.0,0.091443,0.319495,0.290476,0.199147,0.099439,0.757101,0.054814,0.206405,0.233105,0.173030,0.089748,0.242899,0.036629,0.113090,0.057372,0.026118,0.009690


In [222]:
npData = np.around(np.array(dataDf)[:,1:7])
npData

tuningSetAgeRate = [[0.7,0.7,0.9,0.9,0.9,0.9],[26,26,28,28,27,27],[40, 40, 40, 40, 40, 40]]
tuningSetRetireRate = [[1.1, 1.1],[25, 25],[1, 1]]
tuningSet = [tuningSetAgeRate, tuningSetRetireRate]
print(tuningSet)
data = getCost(npData,tuningSet)



tuningList = range(0.1, 1.0, 0.1)



def makeTuningSet(tuningSetAgeRate):
    
    
    return 





[[[0.7, 0.7, 0.9, 0.9, 0.9, 0.9], [26, 26, 28, 28, 27, 27], [40, 40, 40, 40, 40, 40]], [[1.1, 1.1], [25, 25], [1, 1]]]


TypeError: 'float' object cannot be interpreted as an integer

In [40]:
def test():
    npData = np.around(np.array(dataDf)[:,1:7])
    npData

    meanCost = np.array([])
    meanParameter = np.array([])
    resultData = []

    for h in np.arange(0.1,1,0.1):
        for i in np.arange(0.1,1,0.1):
            for j in np.arange(1.1,2.0,0.1):
                for k in np.arange(0.5,1.5,0.1):
                    tuningSetAgeRate = [[h,h,i,i,h,h],[26,26,28,28,27,27],[40, 40, 42, 42, 41, 41]]
                    tuningSetRetireRate = [[j,j],[30, 30],[k, k]]
                    tuningSet = [tuningSetAgeRate, tuningSetRetireRate]
                    data = getCost(npData,tuningSet)
                    resultData.append([[h,i,j,k],data[0],data[1]])
    return resultData
                
%time resultData = test()

Wall time: 2min 43s


In [236]:
npData = np.around(np.array(dataDf)[:,1:7])
npData

meanCost = np.array([])
meanParameter = np.array([])

In [38]:
range(len(resultData))

range(0, 6561)

In [43]:
minCost = [1000000,[]]
for i in range(6561):
    if resultData[i][2][6][2]<minCost[0]:
        minCost = [resultData[i][2][6][2],resultData[i][0]]
        
minCost

[65211.0, [0.9, 0.1, 1.1, 0.7999999999999999]]

[66763.0, [0.9, 0.30000000000000004, 1.1, 0.7000000000000001]]
[66763.0, [0.9, 0.30000000000000004, 1.1, 1.1]]
[70727.0, [0.9, 0.9, 1.1, 0.9]]

[65443.0, [0.9, 0.9, 1.1, 0.30000000000000004]]
[65211.0, [0.9, 0.1, 1.1, 0.7999999999999999]]